<a href="https://colab.research.google.com/github/alondonoco/SenalesSistemas/blob/main/ProyectoFinal/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalación de librerías**

In [1]:
#instalación de librerías
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.8 MB/s eta 0:00:00


In [2]:
!pip install streamlit numpy scipy matplotlib yt-dlp pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.9 MB/s eta 0:00:00


##Crear carpeta pages para trabajar Multiapp en Streamlit

In [3]:
!mkdir pages

# **Página principal**

In [4]:
%%writefile 0_👋_Hello.py

import streamlit as st

st.set_page_config(
    page_title="Proyecto de SyS",
    page_icon="👋",
    layout="wide"
)

st.title("Proyecto Final SyS")

st.sidebar.success("Selecciona una página para explorar.")

st.markdown("""
Bienvenido al desarrollo del Proyecto Final de SyS

---

**Información de los Estudiantes:**

*   **Nombre:** Alejandro Londoño Correa
*   **Cédula:** 1055750510
---
*   **Nombre:** Kevin Loaiza Patiño
*   **Cédula:** 1054478784

"""
)

Writing 0_👋_Hello.py


# **Páginas**

Cada pagina se debe enviar al directorio \pages

#**CONCEPTOS**

In [5]:
%%writefile 1_CONCEPTOS.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.signal import firwin, butter, freqz, lfilter, hilbert

st.title("Conceptos Clave: Teoría y Visualización")

tabs = st.tabs(["Transformada de Fourier", "Filtrado Digital", "Transformada de Hilbert", "Modulación QAM", "OFDM y Comunicaciones Modernas"])

with tabs[0]:
    st.header("Transformada de Fourier")
    st.markdown(r"""
    La **Transformada Discreta de Fourier (DFT)** nos permite ver cómo se descompone una señal en sus frecuencias componentes.
    La **FFT** (Fast Fourier Transform) es una forma eficiente de calcularla.
    La fórmula general es:
    $$ X[k] = \sum_{n=0}^{N-1} x[n] e^{-j \frac{2\pi}{N}kn} $$

    Esta herramienta es fundamental para el análisis espectral en comunicaciones.
    """)

    fs = st.slider("Frecuencia de muestreo [Hz]", 100, 2000, 500)
    f1 = st.slider("Frecuencia 1 [Hz]", 1, fs // 2 - 1, 30)
    f2 = st.slider("Frecuencia 2 [Hz]", 1, fs // 2 - 1, 90)
    T = 1
    t = np.linspace(0, T, int(fs*T), endpoint=False)
    x = np.sin(2*np.pi*f1*t) + 0.5*np.sin(2*np.pi*f2*t)

    X = fft(x)
    freqs = fftfreq(len(x), 1/fs)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 5))
    ax1.plot(t, x)
    ax1.set_title("Señal en el Dominio del Tiempo")
    ax1.set_xlabel("Tiempo [s]")
    ax1.set_ylabel("Amplitud")

    ax2.plot(freqs[:len(freqs)//2], np.abs(X[:len(X)//2]))
    ax2.set_title("Magnitud del Espectro (FFT)")
    ax2.set_xlabel("Frecuencia [Hz]")
    ax2.set_ylabel("|X(f)|")
    ax2.grid()
    st.pyplot(fig)

with tabs[1]:
    st.header("Filtrado Digital FIR / IIR")
    st.markdown(r"""
    El filtrado permite **atenuar** o **preservar** ciertas frecuencias.
    Existen dos tipos principales:
    - **FIR**: siempre estables, buena fase lineal.
    - **IIR**: eficientes, pero pueden volverse inestables.
    """)

    tipo_filtro = st.selectbox("Tipo de filtro", ["FIR (Ventana)", "IIR (Butterworth)"])
    fc = st.slider("Frecuencia de corte (Hz)", 10, fs // 2 - 1, 80)
    ftest = st.slider("Frecuencia de prueba [Hz]", 1, fs // 2 - 1, 30)

    if tipo_filtro == "FIR (Ventana)":
        numtaps = st.slider("Número de coeficientes FIR", 3, 101, 31, step=2)
        b = firwin(numtaps, fc/(fs/2), window='hamming')
        a = [1]
    else:
        orden = st.slider("Orden del filtro IIR", 1, 10, 4)
        b, a = butter(orden, fc/(fs/2))

    x = np.sin(2*np.pi*ftest*t)
    y = lfilter(b, a, x)

    w, h = freqz(b, a, worN=1024, fs=fs)

    fig1, ax = plt.subplots(2, 1, figsize=(8, 5))
    ax[0].plot(t, x, label="Original")
    ax[0].plot(t, y, label="Filtrada")
    ax[0].set_title("Dominio del Tiempo")
    ax[0].legend()
    ax[1].plot(w, 20*np.log10(abs(h)))
    ax[1].set_title("Diagrama de Bode (Magnitud)")
    ax[1].set_xlabel("Frecuencia (Hz)")
    ax[1].set_ylabel("Ganancia (dB)")
    ax[1].grid()
    st.pyplot(fig1)

with tabs[2]:
    st.header("Transformada de Hilbert y Señales Analíticas")
    st.markdown(r"""
    La **Transformada de Hilbert** genera una señal ortogonal (en cuadratura) que tiene un desfase de 90°.
    Esto nos permite crear una **señal analítica**:
    $$ x_a(t) = x(t) + j \cdot \mathcal{H}\{x(t)\} $$
    """)

    f = st.slider("Frecuencia de la señal (Hz)", 1, fs // 2, 50)
    x = np.cos(2*np.pi*f*t)
    xh = hilbert(x)
    q = np.imag(xh)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 5))
    ax1.plot(t, x, label="Señal I (original)")
    ax1.plot(t, q, '--', label="Señal Q (Hilbert)")
    ax1.set_title("Tiempo - Señal I/Q")
    ax1.legend()

    ax2.magnitude_spectrum(xh, Fs=fs)
    ax2.set_title("Magnitud del Espectro (Señal Analítica)")
    st.pyplot(fig)

with tabs[3]:
    st.header("Modulación QAM")
    st.markdown("""
    En QAM (Quadrature Amplitude Modulation), los bits se transforman en símbolos sobre dos señales ortogonales (I y Q).
    La señal transmitida es:
    $$ s(t) = I(t)\\cos(2\\pi f_c t) - Q(t)\\sin(2\\pi f_c t) $$
    """)

    M = st.selectbox("Orden de QAM", [4, 16, 64, 256])
    symbols = np.random.randint(0, M, 200)
    m_side = int(np.sqrt(M))
    I = 2 * (symbols % m_side) - (m_side - 1)
    Q = 2 * (symbols // m_side) - (m_side - 1)

    fig, ax = plt.subplots()
    ax.scatter(I, Q, alpha=0.6)
    ax.set_title(f"{M}-QAM: Diagrama de Constelación")
    ax.set_xlabel("I")
    ax.set_ylabel("Q")
    ax.grid(True)
    st.pyplot(fig)

with tabs[4]:
    st.header("OFDM y Comunicaciones Wi-Fi / 5G")
    st.markdown(r"""
    **OFDM** (Multiplexación por División de Frecuencia Ortogonal) divide la señal en muchas subportadoras ortogonales.
    Cada subportadora usa **QAM**. Esto permite transmitir **múltiples bits por símbolo** eficientemente.

    **Aplicaciones**: Wi-Fi (802.11), 4G LTE, 5G NR.

    ### Diagrama de bloques (Transmisor):
    - Datos → mapeo QAM
    - Agrupación → IFFT
    - Prefijo cíclico
    - DAC → transmisión
    """)

Writing 1_CONCEPTOS.py


In [6]:
!mv 1_CONCEPTOS.py pages/

#**FASE1️⃣**

In [7]:
%%writefile 2_FASE_1️⃣.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import firwin, butter, freqz, lfilter
from scipy.fft import fft, fftfreq

st.title("Fase 1: Análisis en el Dominio de la Frecuencia")

st.markdown("""
Esta fase explora el contenido espectral de una señal y cómo un **filtro paso-bajo** modifica dicho contenido.

Pasos:
1. Generar una señal compuesta por múltiples frecuencias.
2. Visualizarla en el **tiempo** y en el **espectro** (FFT).
3. Aplicar un **filtro paso-bajo digital** (FIR o IIR).
4. Comparar los resultados **antes y después del filtrado**.
5. Analizar la respuesta en frecuencia (diagrama de Bode).

La FFT (Fast Fourier Transform) permite observar la magnitud espectral de una señal en el dominio de la frecuencia:
$X[k] = \sum_{n=0}^{N-1} x[n] e^{-j 2\pi kn/N}$
""")

# Parámetros
fs = st.slider("Frecuencia de muestreo [Hz]", 100, 2000, 500)
T = st.slider("Duración de la señal [s]", 0.1, 2.0, 1.0, step=0.1)
f1 = st.slider("Frecuencia 1 (baja) [Hz]", 5, fs // 2 - 1, 30)
f2 = st.slider("Frecuencia 2 (alta) [Hz]", f1+1, fs // 2 - 1, 90)

# Tiempo y señal original
t = np.linspace(0, T, int(fs*T), endpoint=False)
x = np.sin(2*np.pi*f1*t) + 0.5*np.sin(2*np.pi*f2*t)

# FFT original
X = fft(x)
freqs = fftfreq(len(x), 1/fs)

# Visualización señal original
st.subheader("Señal original en el dominio del tiempo")

fig1, ax1 = plt.subplots()
ax1.plot(t, x)
ax1.set_title("Señal sintética: combinación de 2 frecuencias")
ax1.set_xlabel("Tiempo [s]")
ax1.set_ylabel("Amplitud")
ax1.grid()
st.pyplot(fig1)

# Visualización espectro original
st.subheader("Espectro (FFT) de la señal original")

fig2, ax2 = plt.subplots()
ax2.plot(freqs[:len(freqs)//2], np.abs(X[:len(X)//2]), color="tab:blue")
ax2.set_title("FFT - Magnitud del espectro")
ax2.set_xlabel("Frecuencia [Hz]")
ax2.set_ylabel("Magnitud")
ax2.grid()
st.pyplot(fig2)

# Diseño del filtro
st.subheader("Filtro paso-bajo digital")

filter_type = st.radio("Tipo de filtro", ["FIR (ventana Hamming)", "IIR (Butterworth)"])
cutoff = st.slider("Frecuencia de corte (Hz)", 1, fs // 2 - 1, 40)

if filter_type == "FIR (ventana Hamming)":
    numtaps = st.slider("Número de coeficientes FIR", 5, 101, 31, step=2)
    b = firwin(numtaps, cutoff/(fs/2), window="hamming")
    a = [1.0]
else:
    order = st.slider("Orden del filtro IIR", 1, 10, 4)
    b, a = butter(order, cutoff/(fs/2))

# Aplicar filtro
y = lfilter(b, a, x)

# FFT de señal filtrada
Y = fft(y)

# Comparación en el tiempo
st.subheader("Señal filtrada vs original (tiempo)")

fig3, ax3 = plt.subplots()
ax3.plot(t, x, label="Original", alpha=0.5)
ax3.plot(t, y, label="Filtrada", color="tab:green")
ax3.set_title("Señal original vs filtrada (tiempo)")
ax3.set_xlabel("Tiempo [s]")
ax3.set_ylabel("Amplitud")
ax3.legend()
ax3.grid()
st.pyplot(fig3)

# Comparación en frecuencia
st.subheader("Espectro de la señal filtrada")

fig4, ax4 = plt.subplots()
ax4.plot(freqs[:len(freqs)//2], np.abs(X[:len(X)//2]), label="Original", alpha=0.5)
ax4.plot(freqs[:len(freqs)//2], np.abs(Y[:len(Y)//2]), label="Filtrada", color="tab:green")
ax4.set_title("Espectro antes y después del filtrado")
ax4.set_xlabel("Frecuencia [Hz]")
ax4.set_ylabel("Magnitud")
ax4.legend()
ax4.grid()
st.pyplot(fig4)

# Diagrama de Bode del filtro
st.subheader("Diagrama de Bode: respuesta del filtro")

w, h = freqz(b, a, worN=1024, fs=fs)
fig5, ax5 = plt.subplots()
ax5.plot(w, 20 * np.log10(abs(h)), color="tab:red")
ax5.set_title("Respuesta en frecuencia del filtro")
ax5.set_xlabel("Frecuencia [Hz]")
ax5.set_ylabel("Ganancia [dB]")
ax5.grid()
st.pyplot(fig5)

Writing 2_FASE_1️⃣.py


In [8]:
!mv 2_FASE_1️⃣.py pages/

#**FASE2️⃣**

In [9]:
%%writefile 3_FASE_2️⃣.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from scipy.fft import fft, fftfreq

st.title("Fase 2: Construcción de Señales I/Q usando la Transformada de Hilbert")

st.markdown(r"""
En los sistemas de comunicaciones modernos, las señales **I (en fase)** y **Q (en cuadratura)** son fundamentales para la modulación.

Para obtener estas señales:

- Partimos de una señal real $ x(t) $.
- Aplicamos la **Transformada de Hilbert** para generar una versión ortogonal:
  $$
  \mathcal{H}\{x(t)\} = \text{señal Q, con desfase de } 90^\circ
  $$
- Construimos una **señal analítica compleja**:
  $$
  x_a(t) = x(t) + j \cdot \mathcal{H}\{x(t)\}
  $$
Esta representación es la base de la **modulación I/Q** usada en QAM, OFDM, WiFi y 5G.
""")

# Parámetros interactivos
fs = st.slider("Frecuencia de muestreo [Hz]", 100, 2000, 500)
f0 = st.slider("Frecuencia de la señal (Hz)", 1, fs // 2 - 1, 50)
T = st.slider("Duración de la señal [s]", 0.1, 2.0, 1.0, step=0.1)

# Dominio del tiempo
t = np.linspace(0, T, int(fs*T), endpoint=False)
x = np.cos(2*np.pi*f0*t)                   # Señal original (I)
xa = hilbert(x)                            # Señal analítica
q = np.imag(xa)                            # Señal Q: Hilbert

# 1. Visualización en el tiempo: I(t) y Q(t)
st.subheader("Dominio del tiempo: Señales I(t) y Q(t)")

fig1, ax1 = plt.subplots(figsize=(8, 3.5))
ax1.plot(t, x, label="I(t) = cos(2πf₀t)", color='tab:blue')
ax1.plot(t, q, "--", label="Q(t) = Hilbert{I(t)}", color='tab:orange')
ax1.set_xlabel("Tiempo [s]")
ax1.set_ylabel("Amplitud")
ax1.set_title("Señales I y Q en el dominio del tiempo (desfasadas 90°)")
ax1.grid(True)
ax1.legend()
st.pyplot(fig1)

# 2. Espectro de la señal analítica
st.subheader("Dominio de la frecuencia: Espectro de la señal analítica")

X = fft(xa)
freqs = fftfreq(len(xa), 1/fs)

fig2, ax2 = plt.subplots(figsize=(8, 3.5))
ax2.plot(freqs[:len(freqs)//2], np.abs(X[:len(X)//2]), color='tab:green')
ax2.set_title("Magnitud del espectro de la señal analítica")
ax2.set_xlabel("Frecuencia [Hz]")
ax2.set_ylabel("Magnitud |X(f)|")
ax2.grid()
st.pyplot(fig2)

st.markdown(r"""
La **señal analítica** tiene un **espectro unilateral**, ya que la Transformada de Hilbert elimina las frecuencias negativas.
Esto es clave para las modulaciones I/Q, donde solo se usan frecuencias positivas.
""")

# 3. Curva I/Q paramétrica
st.subheader("Representación paramétrica I-Q (Curva de Lissajous)")

fig3, ax3 = plt.subplots(figsize=(4.5, 4.5))
ax3.plot(x, q, lw=1.2, color='purple')
ax3.set_xlabel("I (Eje real)")
ax3.set_ylabel("Q (Eje imaginario)")
ax3.set_title("Curva I-Q: Señal analítica en el plano complejo")
ax3.grid(True)
ax3.axis("equal")
st.pyplot(fig3)

st.markdown(r"""
Esta curva muestra cómo se comporta la señal $ x_a(t) $ en el plano complejo.
Para una señal cosenoidal pura, la curva es un **círculo** (modulación pura).
Esto confirma que las señales I y Q están perfectamente desfasadas.
""")

Writing 3_FASE_2️⃣.py


In [10]:
!mv 3_FASE_2️⃣.py pages/

#**FASE3️⃣**

In [20]:
%%writefile 4_FASE_3️⃣.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

st.title("Fase 3: Modulación QAM")

st.markdown("""
La **modulación QAM** combina dos señales (I y Q) moduladas en amplitud sobre portadoras ortogonales:

$s(t) = I(t)\cos(2\pi f_c t) - Q(t)\sin(2\pi f_c t)$

Cada símbolo representa varios bits como un punto en el plano I/Q.
Se usa ampliamente en WiFi, 5G y sistemas OFDM.

**Pasos:**
1. Generar bits aleatorios.
2. Mapear a símbolos I y Q.
3. Modular sobre una portadora $f_c$.
4. Visualizar señales, espectros y constelación.
""")

# Parámetros interactivos
M = st.selectbox("Orden de QAM (M)", [4, 16, 64, 256])
fc = st.slider("Frecuencia de portadora [Hz]", 10, 500, 100)
Rb = st.slider("Tasa de símbolo [símb/s]", 10, 1000, 100)
fs = st.slider("Frecuencia de muestreo [Hz]", 200, 5000, 1000)

# Duración de la simulación
Ns = 100  # número de símbolos
Ts = 1 / Rb  # duración por símbolo
T = Ns * Ts  # duración total
t = np.linspace(0, T, int(fs * T), endpoint=False)

# Paso 1: Generar bits aleatorios
k = int(np.log2(M))  # bits por símbolo
bits = np.random.randint(0, 2, Ns * k)

# Paso 2: Mapear a símbolos I y Q (mapa rectangular básico)
symbols = bits.reshape(-1, k)
# Corrected conversion from bits to decimal values
symbols_decimal = np.zeros(Ns, dtype=int)
for i in range(Ns):
  decimal_val = 0
  for j in range(k):
    decimal_val += symbols[i, j] * (2**(k - 1 - j))
  symbols_decimal[i] = decimal_val % M

side = int(np.sqrt(M))
I_vals = 2 * (symbols_decimal % side) - (side - 1)
Q_vals = 2 * (symbols_decimal // side) - (side - 1)

# Interpolación (expandir I y Q en el tiempo)
samples_per_symbol = int(fs / Rb)
I = np.repeat(I_vals, samples_per_symbol)
Q = np.repeat(Q_vals, samples_per_symbol)
t_mod = np.linspace(0, len(I)/fs, len(I), endpoint=False)

# Paso 3: Señal modulada
s = I * np.cos(2*np.pi*fc*t_mod) - Q * np.sin(2*np.pi*fc*t_mod)

# Visualización: I(t), Q(t), s(t)
st.subheader("Señales I(t), Q(t) y señal modulada s(t)")

fig1, ax1 = plt.subplots(3, 1, figsize=(10, 6), sharex=True)
ax1[0].plot(t_mod, I, label="I(t)", color="tab:blue")
ax1[1].plot(t_mod, Q, label="Q(t)", color="tab:orange")
ax1[2].plot(t_mod, s, label="s(t) = señal QAM", color="tab:green")

for ax in ax1:
    ax.grid()
    ax.legend()
    ax.set_ylabel("Amplitud")

ax1[2].set_xlabel("Tiempo [s]")
st.pyplot(fig1)

# Visualización: espectro de s(t)
st.subheader("Espectro de la señal QAM")

S = fft(s)
freqs = fftfreq(len(s), 1/fs)

fig2, ax2 = plt.subplots(figsize=(10, 3))
ax2.plot(freqs[:len(freqs)//2], np.abs(S[:len(S)//2]), color="tab:green")
ax2.set_xlabel("Frecuencia [Hz]")
ax2.set_ylabel("Magnitud")
ax2.set_title("Magnitud del Espectro de s(t)")
ax2.grid()
st.pyplot(fig2)

# Visualización: diagrama de constelación
st.subheader("Diagrama de Constelación (Símbolos I/Q)")

fig3, ax3 = plt.subplots(figsize=(4.5, 4.5))
ax3.scatter(I_vals, Q_vals, color='purple', alpha=0.7)
ax3.set_xlabel("I")
ax3.set_ylabel("Q")
ax3.set_title(f"{M}-QAM: Diagrama de Constelación")
ax3.grid(True)
ax3.axis("equal")
st.pyplot(fig3)

Writing 4_FASE_3️⃣.py


In [21]:
!mv 4_FASE_3️⃣.py pages/

#**FASE4️⃣**

In [13]:
%%writefile 5_FASE_4️⃣.py



Writing 5_FASE_4️⃣.py


In [14]:
!mv 5_FASE_4️⃣.py pages/

#**FASE5️⃣**

In [15]:
%%writefile 6_FASE_5️⃣.py



Writing 6_FASE_5️⃣.py


In [16]:
!mv 6_FASE_5️⃣.py pages/

#**FASE6️⃣**

In [17]:
%%writefile 7_FASE_6️⃣.py



Writing 7_FASE_6️⃣.py


In [18]:
!mv 7_FASE_6️⃣.py pages/

# **Inicialización del Dashboard a partir de túnel local**

1. **Reemplazar nombre de archivo**: Reemplaza el nombre del archivo como se indica en el comentario de la linea 6 de la celda de codigo

2. **Accede al enlace provisional**: Una vez que la aplicación esté corriendo, LocalTunnel generará un enlace temporal. Haz clic o copia ese enlace para acceder a tu aplicación en el navegador (cada vez que corras la celda, el link podrá ser diferente).

**Nota:**
Para finalizar la ejecución del Dashboard ejecuta la ultima celda de codigo y sigue las instrucciones.

In [22]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run 0_👋_Hello.py &>/content/logs.txt & #Cambiar 0_👋_Hello.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-07-16 19:07:14--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64 [following]
--2025-07-16 19:07:14--  https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/37d2bad8-a2ed-4b93-8139-cbb15162d81d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-16T19%3A44%3A08Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-16T1